# Rivulet: USGS Water Data APIs
_by Michelle H Wilkerson_

## Purpose of this Notebook

This notebook was developed as part of NSF Grant 2445609 to support accessing and processing public data for middle and high school classroom activities. It's written to be relatively accessible to beginners, but if you have not interacted with computational notebooks or python before you may find navigating this tool difficult. (Check out the Show Your Work project for a gentle introduction to computational notebooks for educators!)

Our project is focused on supporting data analysis and mechanistic reasoning in science education. In other words, we want students to learn how data provides information about _how scientific mechanisms work_, and how understanding scientific mechanisms can help them to _explain and interpret patterns in data_. This builds on a long history of research on complex systems and agent-based modeling, and more closely connects that work to current expansions of data analysis across subjects.

Here, we are focused on Water Quality as a phenomenon. While most students understand that poor Water Quality can impact health, they may not know what sorts of pollutants impact water quality, and what kinds of events or conditions lead to reductions in water quality.

This data tool allows users to connect to the United States Geological Survey (USGS) water data APIs, search for water quality data streams in an area of interest, and then provides a collection of ways to fitler and search the data to ensure you find datasets that have patterns worth exploring. These kinds of datasets can serve as a launch to examining what WQ is and what are its underlying mechanistic and compositional complexities.

You are welcome to modify and adapt this script. You may find the USGS' water data APIs documentation [here](https://api.waterdata.usgs.gov/) and [here]() helpful.

## Part 0: What is an API? (Click to expand...)

Describe APIs, how they work, how common they are in data science, why they are useful for educators and educational researchers who do data science education work to know about. Describe the risks and concerns about APIs and using them to source data that students will interact with.

## Part I: Connecting with USGS Water Data APIs

You can sign up for an API key [at this site](https://api.waterdata.usgs.gov/signup). Once you recieve it, replace the DEMO_KEY below with your unique API key. Do not share your key!

In [ ]:
API_KEY = "DEMO_KEY" 

## Part 2: Specifying a Location and Time Period

We likely found more locations than we expected! But, these are not all likely to have the specific information we need. Let's filter our results so we're only looking at locations that PM2.5 and O3.

Good! Now you can clearly see the id of stations that have the data you need. You can also see the interval of times for which these records are available by scrolling all the way to the right in the table above. If you're using the originally hard-coded bay area, these look like they both started in 2016 and are up-to-date.

Ok so our O3 sensor id is 3833, and our PM2.5 sensor is 3835. Let's fetch the latest data for each of these sensors. We clean it up and merge it into one dataset.

Here's some code to expore a csv of the merged dataset to use in other tools. (Should be very easy to work with in CODAP from here.)